# Наивный Байес

## Теорема Байеса
$ P(A|B) = \frac{P(B|A) P(A)}{P(B)} $, где

$P(B|A)$ — вероятность наступления события B при истинности гипотезы A;

$P(A)$ — априорная вероятность гипотезы A;

$P(B)$ —  полная вероятность наступления события B;

$P(A|B)$ —  априорная вероятность гипотезы A.


## Наивный байесовский классификатор
$P(C|D) = \frac{P(D|C) P(C)}{P(D)}$, где

$P(C|D)$ — вероятность что документ $D$ принадлежит классу $C$;

$P(D|C)$ — вероятность встретить документ $D$ среди всех документов класса $C$;

$P(C)$ — безусловная вероятность встретить документ класса $C$ в корпусе документов (упр. список всех слов);

$P(D)$ — безусловная вероятность документа $D$ в корпусе документов.

Вероятность $P(D|C)$ можно найти с помощью произведения вероятностей всех слов входящих в документ:
$P(D|C) \approx \prod_{i=1}^n P(w_{i}|C)$, где

$n$ — количество слов;

$w_{i}$ — слово $w$ с индексом $i$;

$P(w_{i}|C)$ — оценка вероятности встретить слово $w_{i}$ в классе $С$. Существуют различные методы оценки. Например, multinomial bayes model:

$P(w_{i}|C) = \frac{W_{ic}}{W_{c}}$, где

$W_{ic}$ — количество вхождения $i$-го слова в документы класса $C$

$W_{c}$ — словарь корпуса документов (список всех уникальных слов)

Простыми словами:
-числитель описывает сколько раз слово встречается в документах класса (включая повторы),
-знаменатель – это суммарное количество слов во всех документах этого класса.

$P(C) = \frac{D_{c}}{D}$, где

$D_{c}$ — количество документов принадлежащих классу $C$

$D$ — общее количество документов в обучающей выборке

$P(D) = \sum_{i=1}^n P(D|C_{i}) P(C_{i})$, ее мы редуцируем т.к. значение неизменно от документа к документу

Итоговая формула:

$P(C|D) = \prod_{i=1}^n \frac{W_{ic}}{\sum_{i \in V} W_{ic}} \frac{D_{c}}{D} = \frac{D_{c}}{D} \prod_{i=1}^n \frac{W_{ic}}{W_{c}}$

### Реализуем алгоритм

In [391]:
import pandas as pd
import numpy as np
import string
import nltk
from numpy import e
from collections import Counter
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/mitya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [392]:
# создадим словарь корпуса документов
docs_corp = pd.DataFrame()
docs_corp['spam_0'] = ['купить', 'виагра', 'дешево']
docs_corp['spam_1'] = ['консультация', 'тренер', 'бесплатно']
docs_corp['ham_0'] = ['купить', 'молоко', 'магазин']
docs_corp

,spam_0,spam_1,ham_0
0,купить,консультация,купить
1,виагра,тренер,молоко
2,дешево,бесплатно,магазин


In [393]:
class_docs = pd.DataFrame()
class_docs['spam'] = [len(docs_corp.filter(like='spam').columns), docs_corp.filter(like='spam').count().sum()]
class_docs['ham'] = list(docs_corp.filter(like='ham').shape)[::-1]
class_docs

,spam,ham
0,2,1
1,6,3


In [394]:
D = len(docs_corp.columns)
D

3

In [395]:
spam = pd.DataFrame(np.unique(docs_corp.filter(like='spam').values, return_counts=True), index=['words', 'spam']).T.set_index(keys=['words'])
ham = pd.DataFrame(np.unique(docs_corp.filter(like='ham').values, return_counts=True), index=['words', 'ham']).T.set_index(keys=['words'])
words_corp = pd.concat([spam, ham], axis=1).fillna(0)
words_corp

,spam,ham
words,,
бесплатно,1,0
виагра,1,0
дешево,1,0
консультация,1,0
купить,1,1
тренер,1,0
магазин,0,1
молоко,0,1


In [396]:
P_C_spam = class_docs.iat[0, 0] /  D
P_C_spam

0.6666666666666666

In [397]:
P_C_ham = class_docs.iat[0, 1] /  D
P_C_ham

0.3333333333333333

In [398]:
# фраза для классификации
need_to_class = ['купить', 'молоко']

Вспомним формулу
$P(C|D) =  \frac{D_{c}}{D} \prod_{i=1}^n \frac{W_{ic}}{W_{c}}$

In [399]:
P_DC_spam = 1
P_DC_ham = 1

for word in need_to_class:
    P_DC_spam *= words_corp.loc[word][0] / class_docs.iat[1, 0]
    P_DC_ham *= words_corp.loc[word][1] / class_docs.iat[1, 1]

P_CD_spam = P_C_spam * P_DC_spam
P_CD_ham = P_C_ham * P_DC_ham
print(P_CD_spam)
print(P_CD_ham)

0.0
0.037037037037037035


Похоже у нас проблема. Из-за того, что слово 'молоко' не встречается в списке слов из категории спам мы получаем в качестве результата 0.
Окей, давайте немного изменим формулу, чтобы решить проблему. Увеличим счетчик каждого слова на 1, вот что получается:

$P(w_{i}|C) = \frac{W_{ic} + 1}{W_{c} + 1}$


In [400]:
def naive_bayes(need_to_class):
    P_DC_spam = 1
    P_DC_ham = 1

    for word in need_to_class:
        P_DC_spam *= (words_corp.loc[word][0] + 1) / (class_docs.iat[1, 0] + 1)
        P_DC_ham *= (words_corp.loc[word][1] + 1) / (class_docs.iat[1, 1] + 1)

    P_CD_spam = P_C_spam * P_DC_spam
    P_CD_ham = P_C_ham * P_DC_ham
    return P_CD_spam, P_CD_ham

P_CD_spam, P_CD_ham = naive_bayes(need_to_class)
print(P_CD_spam, P_CD_ham)

0.027210884353741492 0.08333333333333333


In [401]:
need_to_class = ['купить', 'виагра']
P_CD_spam, P_CD_ham = naive_bayes(need_to_class)
print(P_CD_spam, P_CD_ham)

0.054421768707482984 0.041666666666666664


Отлично, наш классификатор работает. Давайте избавимся от еще одной возможной проблемы. Если документ будет большим, нам придется перемножать огромное количество небольших чисел, что в итоге приведет к арифметическому переполнению снизу. Воспользуемся свойством произведения логарифма:
$log(ab) = log(a)+ log(b)$

Модифицируем нашу формулу:

$ P(C|D) = log \frac{D_{c}}{D} + \sum_{i=1}^n log \frac{W_{ic}}{W_{c}} $

In [402]:
P_C_spam = np.log(class_docs.iat[0, 0] /  D)
P_C_ham = np.log(class_docs.iat[0, 1] /  D)


def naive_bayes_class(need_to_class):
    P_DC_spam = 1
    P_DC_ham = 1

    for word in need_to_class:
        P_DC_spam += np.log((words_corp.loc[word][0] + 1) / (class_docs.iat[1, 0] + 1))
        P_DC_ham += np.log((words_corp.loc[word][1] + 1) / (class_docs.iat[1, 1] + 1))

    P_CD_spam = P_C_spam + P_DC_spam
    P_CD_ham = P_C_ham + P_DC_ham
    return P_CD_spam, P_CD_ham

In [403]:
need_to_class = ['купить', 'молоко']
P_CD_spam, P_CD_ham = naive_bayes_class(need_to_class)
print(P_CD_spam, P_CD_ham)

-2.604138225658846 -1.4849066497880004


In [404]:
train_dataset = pd.read_csv('/Users/mitya/Downloads/spam.csv', encoding='ISO-8859-1')

In [405]:
train_dataset = train_dataset[['v1', 'v2']]

In [406]:
train_dataset.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [407]:
class_docs = pd.DataFrame()
class_docs['D'] = train_dataset['v1'].value_counts()
class_docs

,D
ham,4825
spam,747


In [408]:
def text_process(mess):
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

In [409]:
train_dataset['v3'] = train_dataset.v2.apply(text_process)

In [410]:
words = train_dataset[train_dataset.v1=='ham'].v3.apply(lambda x: [word.lower() for word in x.split()])
ham_words = Counter()

for msg in words:
    ham_words.update(msg)


words = train_dataset[train_dataset.v1=='spam'].v3.apply(lambda x: [word.lower() for word in x.split()])
spam_words = Counter()

for msg in words:
    spam_words.update(msg)

In [411]:
df = pd.DataFrame.from_dict(ham_words, orient='index')
df.columns = ['spam']

In [412]:
spam = pd.DataFrame.from_dict(spam_words, orient='index')
spam.columns = ['spam']
ham = pd.DataFrame.from_dict(ham_words, orient='index')
ham.columns = ['ham']
words_corp = pd.concat([spam, ham], axis=1).fillna(0)
words_corp.head()

,spam,ham
free,216.0,59.0
entry,26.0,0.0
wkly,14.0,0.0
comp,10.0,1.0
win,60.0,11.0


In [413]:
D = int(class_docs.sum())
D

5572

In [414]:
P_C_spam = np.log(class_docs.iloc[1] /  D)
P_C_ham = np.log(class_docs.iloc[0] /  D)


def naive_bayes_class_final(need_to_class):
    P_DC_spam = 1
    P_DC_ham = 1

    for word in need_to_class:
        P_DC_spam += np.log((words_corp.loc[word][0] + 1) / (class_docs.iloc[1] + 1))
        P_DC_ham += np.log((words_corp.loc[word][1] + 1) / (class_docs.iloc[0]  + 1))

    P_CD_spam = P_C_spam + P_DC_spam
    P_CD_ham = P_C_ham + P_DC_ham
    return P_CD_spam, P_CD_ham

In [415]:
need_to_class = ['still', 'ok', 'call', 'next', 'day']

P_CD_spam, P_CD_ham = naive_bayes_class_final(need_to_class)
print('P_CD_spam: ', float(P_CD_spam))
print('P_CD_ham: ', float(P_CD_ham))

P_CD_spam:  -18.7672534826609
P_CD_ham:  -16.428236296624597


Теперь давайте получим вероятности. Для этого избавимся от логарифмов и нормируем значения:

$P(C|D) = \frac{e^{q_{c}}}{\sum e^{q_{c}}}$, где
$q_{c}$ — логарифмическая оценка

In [416]:
spam_proba = e**P_CD_spam / (e**P_CD_spam + e**P_CD_ham)
ham_proba = e**P_CD_ham / (e**P_CD_spam + e**P_CD_ham)
print('spam_proba: ', float(spam_proba))
print('ham_proba', float(ham_proba))

spam_proba:  0.0879427132327269
ham_proba 0.9120572867672732
